# 0. Importing packages

In [7]:
print("Importing packages")
# import pandas as pd
# import numpy as np
# import uproot
# import awkward as ak
# import glob
import random
from scipy.optimize import differential_evolution
import pickle
import time
from matplotlib import pyplot as plt
from packages.functions import *
import optimisation_2024 as opt
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

global CPUs
CPUs = 30 #cpu_count()


Importing packages


# 1. Loading data

In [134]:
data2024 = "/shared/scratch/wq22321/data/NANOAOD_Muon0Run2023D_ZMu_PromptReco_v2RAW_RECO_2023_v0_4/231121_100830/0000/out_*.root"

In [135]:
fit, valid = opt.prepareInputs(dir = data2024, subset=0.7, cuts=(0, 250))

Reading in calo tower data
Reading in PUPPI MET pT and phi
Calculating x and y components of PUPPI MET
Reading in muon pT and phi
Calculating muon ptx and pty for each event
Calculating PUPPI MET no Mu and reformatting data
Flattening MET distribution


/users/wq22321/methackathon_2024/packages/readData.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pfcand_muons['muon_ptx'] = np.cos(pfcand_muons['Muon_phi']) * pfcand_muons['Muon_pt']
/users/wq22321/methackathon_2024/packages/readData.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pfcand_muons['muon_pty'] = np.sin(pfcand_muons['Muon_phi']) * pfcand_muons['Muon_pt']


ValueError: too many values to unpack (expected 3)

In [ ]:
puppi, calo, ntt4 = fit

# 3. Optimisation

In [ ]:
print("Starting optimisation")

bounds = [(0, 4), (0, 4), (0, 4), (0, 4)]
x0 = (2.0, 2.0, 0.5, 2.0)
result = differential_evolution(
    func     = opt.objective,
    bounds   = bounds,
    popsize  = 40,
    maxiter  = 100,
    strategy = "best1bin",
    #init     = "sobol",
    disp     = True,
    workers  = 2,
    polish   = False
)

print(result.x)
print(result)

# 4. Parameter analysis

In [ ]:
def heatmapper(a, b, c, d):
    dat = lookup_gen(a, b, c, d)
    ietas, pu_bins, thresh = zip(*dat)
    plt.figure(figsize=(15,9))
    plt.hist2d(ietas, pu_bins, weights=thresh, bins=[83, 32])
    plt.colorbar()
    #plt.xlim([1,41]); plt.ylim([0,31])
    plt.xlabel("ieta"); plt.ylabel("NTT4 bin")
    plt.title("2D histogram of tower threshold (scaled by tower size)")
    plt.show()

In [ ]:
def lookup_gen(a, b, c, d):
    
    all_ieta_vals = np.linspace(-41, 41, 83)
    all_pu_bins = np.linspace(0, 31, 32)
    
    res = []
    for ieta in all_ieta_vals:
        for pu_bin in all_pu_bins:
            thresh = threshold_calc(ieta, pu_bin, a, b, c, d)
            res.append((ieta, pu_bin, thresh))
            
    return res

In [ ]:
def threshold_calc(ieta, ntt4, a, b, c, d):
    towerAreas = [    0., # dummy for ieta=0
                  1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,
                  1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,
                  1.03,1.15,1.3,1.48,1.72,2.05,1.72,4.02,
                  0., # dummy for ieta=29
                  3.29,2.01,2.02,2.01,2.02,2.0,2.03,1.99,2.02,2.04,2.00,3.47]
    
    numerator = (towerAreas[int(abs(ieta))]**a) * (ntt4**c)
    denominator = d * (1 + np.exp(-b * (abs(ieta))))
    
    threshold = (numerator / denominator).clip(max=40)
#    return (threshold/2)# / towerAreas[int(abs(ieta))]
    if towerAreas[int(abs(ieta))] == 0:
        return np.nan
    else:
        return (threshold/2)# / towerAreas[int(abs(ieta))]

In [ ]:
fw = (0.66881105, 1.77791976, 0.96226907, 2.08912586)
reg = (0.21456065, 2.74961211, 0.47896413, 0.30650943)
old = (1.707, 3.078, 0.195, 1.365)

zmu23_1 = (1.87, 1.34, 0.1, 2.84)
zmu23_2 = (1.49, 2.91, 0.26, 1.51)
zmu23_3 = (2.34, 2.82, 0.37, 2.79)

In [ ]:
heatmapper(*zmu23_2)

In [ ]:
valid_calo, valid_puppi, valid_ntt4 = validOld

In [ ]:
MET = applyCaloTowerThresh(valid_calo, *zmu23_2)[0]

In [ ]:
l1met = calcL1MET(valid_calo)

In [ ]:
def getTurnOn( online, offline, threshold=80 ) :
    offline_bins = np.linspace(0, 300, 40)
    efficiency = []


    for i in range(len(offline_bins) - 1):
        # Define the offline range for this bin
        offline_range = (offline >= offline_bins[i]) & (offline < offline_bins[i + 1])
        # count the number of events passing the threshold in the offline range
        num_offline = sum(offline_range)
        # count the number of events passing the threshold in both online and offline ranges
        num_both = sum((online > threshold) & offline_range)
        # calculate the efficiency as the ratio of online events passing the cut over offline events passing the threshold
        if num_offline > 0:
            eff = num_both / num_offline
        else:
            eff = 0
        efficiency.append(eff)

    bin_centers = (offline_bins[:-1] + offline_bins[1:]) / 2

    return bin_centers, efficiency

In [ ]:
bins, turnon = getTurnOn(MET, valid_puppi)

In [ ]:
_, turnon_noPU = getTurnOn(l1met, valid_puppi)

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(bins, turnon, label="With PU sup")
plt.plot(_, turnon_noPU, label = "No PU sup")

plt.grid(True, linestyle='--', color='gray', alpha=0.5)
plt.hlines(0.95, xmin=0, xmax=300, colors='gray', linestyles='--', alpha=0.5)
plt.legend()
plt.show()

# Reading old data